# Detect Objects in Images

**Warning:** This notebook assumes that you have run the Object Detection Workspace Setup notebook and have a configured workspace

### Resources

https://github.com/nicknochnack/TFODCourse <br>
**Highly Recommend Watching the Tensorflow Object Detection Walkthrough Video**

## 1. Import Libraries

In [ ]:
import os

## 2. Setup Workspace Paths & Specify Model to use

In [ ]:
#Set true if using a custom model
MODEL_TYPE = False
#Set to the name of the custom model folder or pre-trained model folder
MODEL_NAME = 'centernet_resnet50_v1_fpn_512x512_coco17_tpu-8'
#Sets Label Category - Determined by training dataset
LABEL_MAP_NAME = 'mscoco_label_map.pbtxt'
#Set project location and directory name
PROJECT_NAME = "C:\\Users\Richard\\Desktop\\TensorflowTest2\\"


if MODEL_TYPE:
    
    paths = {
        'MODEL_PATH': os.path.join(PROJECT_NAME, 'workspace','custom_models'),
        'CHECKPOINT_PATH': os.path.join(PROJECT_NAME, 'workspace','custom_models',MODEL_NAME,'final_model','checkpoint')
    }
    
    files = {
        'PIPELINE_CONFIG':os.path.join(PROJECT_NAME, 'workspace','custom_models', MODEL_NAME,'final_model','pipeline.config'),
        'LABELMAP': os.path.join(paths['MODEL_PATH'], 'label_map', LABEL_MAP_NAME)
    }
    
else:
    
    paths = {
        'MODEL_PATH': os.path.join(PROJECT_NAME, 'workspace','pre-trained-models'),
        'CHECKPOINT_PATH': os.path.join(PROJECT_NAME, 'workspace','pre-trained-models',MODEL_NAME,'checkpoint')
    }

    files = {
        'PIPELINE_CONFIG':os.path.join(PROJECT_NAME, 'workspace','pre-trained-models', MODEL_NAME, 'pipeline.config'),
        'LABELMAP': os.path.join(PROJECT_NAME, 'models', 'research', 'object_detection', 'data', 'mscoco_label_map.pbtxt')
    }

## 3. Import Libraries Needed to Run Model

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import cv2 
import glob
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

## 4. Set Pipeline Config & Create Model from Saved Checkpoint

In [ ]:
#Set checkpoint file to load
checkpoint = 'ckpt-0'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], checkpoint)).expect_partial()

#Sets the pbtxt file to use - specified by LABEL_MAP_NAME variable 
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

## 5. Specify Directory with Images for Object Detection

In [ ]:
#Set path to images for object detection
IMAGES_PATH = "C:\\Users\\Richard\\Desktop\\TensorflowTest2\\workspace\\images\\train"
#Set image file type
IMAGE_TYPE = ".jpg"

IMAGES = glob.glob(os.path.join(IMAGES_PATH, '*' + IMAGE_TYPE))

#Sets image to do object detection on
IMAGE_PATH = ("C:\\Users\Richard\\Desktop\\thumbsup.cd56334e-ff92-11ec-9566-84144db13b80.jpg")

## 6. Create Method that Performs Object Detection on one Image

In [ ]:
def detect_obj(image_path):
    img = cv2.imread(image_path)
    image_np = np.array(img)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
                image_np_with_detections,
                detections['detection_boxes'],
                detections['detection_classes']+label_id_offset,
                detections['detection_scores'],
                category_index,
                use_normalized_coordinates=True,
                max_boxes_to_draw=5,
                min_score_thresh=.8,
                agnostic_mode=False)

    plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    plt.show()

## 7. Call Method for each Image in Specified Directory

In [ ]:
for image_path in IMAGES:
    detect_obj(image_path)